In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from ops import *
import cv2
import os

import numpy as np
np.random.seed(1234)
import matplotlib.pyplot as plt
import tensorflow as tf
import time